# Collaboration recommender system - Baseline

## **Setting up environment**

---



### **Package installation**

Installing `torch` and `torch_geometric` libraries.

In [ ]:
!pip3 install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu121
# Installation due to error: Faiss assertion 'err == CUBLAS_STATUS_SUCCESS' failed
!pip install https://github.com/kyamagu/faiss-wheels/releases/download/v1.7.3/faiss_gpu-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 432.4/432.4 MB 6.3 MB/s eta 0:00:00


In [ ]:
import torch
import os
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.4.0+cu121


In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install pyg-lib -f https://data.pyg.org/whl/nightly/torch-${TORCH}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
Looking in links: https://data.pyg.org/whl/torch-2.4.0+cu121.html
Looking in links: https://data.pyg.org/whl/nightly/torch-2.4.0+cu121.html
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-kj4kzg9o
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-kj4kzg9o
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit f5c829344517c823c24abb08ce2fc7cf00ff29f7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### **Loading libraries**

In [19]:
# PyTorch imports
from torch import Tensor
import torch.nn.functional as F
from torch.nn import Linear, BCEWithLogitsLoss

# PyTorch Geometric imports
import torch_geometric
import torch_geometric.transforms as T
from torch_geometric.loader import LinkNeighborLoader, NeighborLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn.models.lightgcn import BPRLoss, LightGCN
from torch_geometric.data import Data
from torch_geometric.utils import structured_negative_sampling
from torch_geometric.metrics import (
    LinkPredPrecision,
    LinkPredRecall,
    LinkPredMAP,
    LinkPredMRR,
    LinkPredNDCG
    )

# Other imports
import io
import datetime
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
from google.cloud import bigquery
from google.cloud import storage

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.cuda.get_device_name(0))
print(torch.__version__)
print(torch_geometric.__version__)
print(f"Device: '{device}'")

NVIDIA A100-SXM4-40GB
2.4.0+cu121
2.7.0
Device: 'cuda'


### **Global variables**

In [ ]:
# Initiate global variables
bq_client = bigquery.Client()
storage_client = storage.Client()

num_recommendations = 5 # Number of recommendations
num_train = 0.8 # Percentage of data used for training
learning_rate = 1e-2 # Learning rate
num_epochs = 1000 # Number of epochs
hidden_channels = 128 # Number of hidden channels


## **Data preparation**

---



### **Loading data**

When loading the data, we take into account only the articles, where at least one author comes from the EUTOPIA

In [ ]:
# Get all authors data and value metrics about their collaboration
author_query = f"""
SELECT AUTHOR_SID,
       PUBLICATION_COUNT
FROM PROD.V_GRAPH_V3_NODE_AUTHOR
"""
author_df = bq_client.query(author_query).to_dataframe()


# Get all edges between authors and co-authors
coauthored_query = f"""
SELECT AUTHOR_SID,
       CO_AUTHOR_SID,
       TIME
FROM PROD.V_GRAPH_V3_EDGE_CO_AUTHORS
"""
coauthored_df = bq_client.query(coauthored_query).to_dataframe()

### Contiguous unique identifier for node: **author**

In [ ]:
# Author: Map each unique MD5 hash to a contiguous unique integer ID
unique_authors = author_df['AUTHOR_SID'].unique()
author_id_map = {author: i for i, author in enumerate(unique_authors)}
author_sid_map = {y: x for x, y in author_id_map.items()}
# ---> Adjust all dataframes
author_df['AUTHOR_NODE_ID'] = author_df['AUTHOR_SID'].map(author_id_map)
coauthored_df['AUTHOR_NODE_ID'] = coauthored_df['AUTHOR_SID'].map(author_id_map)
coauthored_df['CO_AUTHOR_NODE_ID'] = coauthored_df['CO_AUTHOR_SID'].map(author_id_map)

## **Homogeneous graph creation**
First of all, we prepare the node features for articles. We first sort the article dataframe by node ID. We know that we have unique values in the article dataframe, i.e. one row per article and we can just sort it. Otherwise, we would need to create a unique mapping between article features and articles themselves. The sorting needs to match the node index that we will create later. After that, we also need to set up correct type (specifically, convert Pandas Int64 to int64, but we go for the lazy version and just convert all features to float64). At last, we exclude the `ARTICLE_SID` and `ARTICLE_NODE_ID` columns, because Torch can't work with strings.

**TODO:**
- Think about only including "valuable" partnerships/edges.

### Matrix X for node: **article**

In [ ]:
# Article X
# Sort article dataframe
co_authors_edge_attr_columns = list(filter(lambda x: x not in ('AUTHOR_SID', 'CO_AUTHOR_SID', 'AUTHOR_NODE_ID', 'CO_AUTHOR_NODE_ID', 'TIME'), coauthored_df.columns))

# Convert types
edge_attr_co_authors = coauthored_df[co_authors_edge_attr_columns].astype('float64').values

### Matrix X for node: **author**

In [ ]:
# Author X
# Sort author dataframe
sorted_author_df = author_df.sort_values(by='AUTHOR_NODE_ID')
# Exclude columns AUTHOR_SID, AUTHOR_NODE_ID
author_x_columns = list(filter(lambda x: x not in ('AUTHOR_SID', 'AUTHOR_NODE_ID'), sorted_author_df.columns))

# Convert types
author_x = sorted_author_df[author_x_columns].astype('float64').values

### Edge index for edge: **(author, co_authors, author)**

In [ ]:
# Add edge index: for edges corresponding to authors co-authoring articles (author to author connection)
author_node_ids = torch.from_numpy(coauthored_df['AUTHOR_NODE_ID'].values)
coauthor_node_ids = torch.from_numpy(coauthored_df['CO_AUTHOR_NODE_ID'].values)
edge_index_co_authors = torch.stack([author_node_ids, coauthor_node_ids], dim=0)
edge_time_co_authors = torch.from_numpy(np.array(coauthored_df['TIME'].values.astype('int64')))

### Data object

After generating the initial node feature Numpy array, we create an instance of `HeteroData` class with two types of nodes corresponding to authors and articles and an edge denoting authors publishing articles.

*Note: We also need to make sure to add the reverse edges from authors to aritcles in order to let a GNN be able to pass messages in both directions. We can leverage the `T.ToUndirected()` transform for this from PyG.*

In [ ]:
def split_to_train_and_test(data, num_train=0.8):
    time = data.edge_time
    perm = time.argsort()
    train_index = perm[:int(num_train * perm.numel())]
    test_index = perm[int(num_train * perm.numel()):]

    # Edge index
    data.train_pos_edge_index = data.edge_index[:, train_index]
    data.test_pos_edge_index = data.edge_index[:, test_index]

    # Add negative samples to test
    neg_edge_index_i, neg_edge_index_j, neg_edge_index_k = structured_negative_sampling(
        edge_index=data.test_pos_edge_index,
        num_nodes=data.num_nodes
        )
    data.test_neg_edge_index = torch.stack([neg_edge_index_i, neg_edge_index_k], dim=0)

    # data.edge_index = data.edge_attr = data.edge_time = None
    return data

In [ ]:
data = Data()

# Save node indices:
data.node_id = torch.arange(len(unique_authors))
# Add edge 'co_authors'
data.edge_index = edge_index_co_authors
data.edge_attr = torch.from_numpy(edge_attr_co_authors).to(torch.float)
data.edge_time = edge_time_co_authors

# Set X for author nodes
data.x = torch.from_numpy(author_x).to(torch.float)

# Metadata about number of features and nodes
data.num_features = data.x.shape[1]
data.num_nodes = data.x.shape[0]

# Feature normalization
data = split_to_train_and_test(data)

data

Data(node_id=[56233], edge_index=[2, 252269], edge_attr=[252269, 0], edge_time=[252269], x=[56233, 1], num_features=1, num_nodes=56233, train_pos_edge_index=[2, 201815], test_pos_edge_index=[2, 50454], test_neg_edge_index=[2, 50454])

## Model training


---



In [23]:
# Initialize the model
model = LightGCN(num_nodes=data.num_nodes,
    embedding_dim=hidden_channels,
    num_layers=2).to(device)

# Transfer to device
data = data.to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, mode='min', factor=0.5, patience=5, verbose=True)
criterion_bcewll = BCEWithLogitsLoss().to(device)

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [22]:
def train():
    model.train()

    pos_edge_index = data.train_pos_edge_index
    # Negative sampling
    neg_edge_index_i, neg_edge_index_j, neg_edge_index_k = structured_negative_sampling(
        edge_index=pos_edge_index,
        num_nodes=data.num_nodes)
    neg_edge_index = torch.stack([neg_edge_index_i, neg_edge_index_k], dim=0)

    optimizer.zero_grad()

    edge_label_index = torch.cat([
        pos_edge_index,
        neg_edge_index,
    ], dim=1)

    pos_rank, neg_rank = model(pos_edge_index, edge_label_index).chunk(2)

    # Calculate BPR loss
    loss = loss_bpr = model.recommendation_loss(
        pos_rank,
        neg_rank,
        node_id=edge_label_index.unique(),
    )
    loss.backward()
    optimizer.step()

    total_loss = float(loss) * pos_rank.numel()
    total_examples = pos_rank.numel()

    # Cleanup
    del pos_rank, neg_rank
    torch.cuda.empty_cache()


    return  total_loss / total_examples


@torch.no_grad()
def test():
    model.eval()

    pos_edge_index = data.test_pos_edge_index
    neg_edge_index = data.test_neg_edge_index

    edge_label_index = torch.cat([
        pos_edge_index,
        neg_edge_index,
    ], dim=1)

    optimizer.zero_grad()
    pos_rank, neg_rank = model(pos_edge_index, edge_label_index).chunk(2)

    loss = model.recommendation_loss(
        pos_rank,
        neg_rank,
        node_id=edge_label_index.unique(),
    )

    total_loss = float(loss) * pos_rank.numel()
    total_examples = pos_rank.numel()

    # Cleanup
    del pos_rank, neg_rank
    torch.cuda.empty_cache()

    return total_loss / total_examples

@torch.no_grad()
def evaluate(k:int=20):
    model.eval()
    embs = model.get_embedding(data.train_pos_edge_index).to(device)
    recalls = []

    result = {
        'precision@k': LinkPredPrecision(k=k).to(device),
        'recall@k': LinkPredRecall(k=k).to(device),
        'map@k': LinkPredMAP(k=k).to(device),
        'mrr@k': LinkPredMRR(k=k).to(device),
        'ndcg@k': LinkPredNDCG(k=k).to(device)
        }

    # Calculate distance between embeddings
    logits = embs @ embs.T

    # Exclude training edges
    logits[data.train_pos_edge_index[0], data.train_pos_edge_index[1]] = float('-inf')

    # Gather ground truth data
    ground_truth = data.test_pos_edge_index

    # Get top-k recommendations for each node
    top_k_index = torch.topk(logits, k=k, dim=1).indices

    # Update performance metrics
    for metric in result.keys():
      result[metric].update(
          pred_index_mat=top_k_index,
          edge_label_index=ground_truth)

    # Cleanup
    del embs, logits, ground_truth, top_k_index
    torch.cuda.empty_cache()

    return result

In [24]:
results = []
for epoch in range(1,  200):
    train_loss = train()
    test_loss = test()
    scheduler.step(test_loss)
    eval_result = evaluate(k=num_recommendations)

    # Save results
    epoch_result = {
        'epoch': epoch,
        'train_loss': train_loss,
        'test_loss': test_loss,
        'precision@k': eval_result['precision@k'].compute(),
        'recall@k': eval_result['recall@k'].compute(),
        'map@k': eval_result['map@k'].compute(),
        'mrr@k': eval_result['mrr@k'].compute(),
        'ndcg@k': eval_result['ndcg@k'].compute()
    }
    results.append(epoch_result)

    # Log results
    if epoch % 50 == 0:
        # Log model performance
        formatted_str = ', '.join([f'{key}: {epoch_result[key]:.4f}' for key in epoch_result.keys()])
        print(formatted_str)


results = pd.DataFrame(results)

epoch: 50.0000, train_loss: 0.0204, test_loss: 0.2463, precision@k: 0.0269, recall@k: 0.0330, map@k: 0.0280, mrr@k: 0.0551, ndcg@k: 0.0393
epoch: 100.0000, train_loss: 0.0061, test_loss: 0.2157, precision@k: 0.0294, recall@k: 0.0365, map@k: 0.0313, mrr@k: 0.0634, ndcg@k: 0.0439
epoch: 150.0000, train_loss: 0.0033, test_loss: 0.2082, precision@k: 0.0306, recall@k: 0.0382, map@k: 0.0334, mrr@k: 0.0676, ndcg@k: 0.0465


### Model evaluation

In [ ]:
# Generate loss curve
plt.plot(results['epoch'], results['train_loss'], label='train')
plt.plot(results['epoch'], results['test_loss'], label='test')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.show()

In [ ]:
# Generate evaluation metrics plot
plt.plot(results['epoch'], results['precision@k'], label='precision@k')
plt.plot(results['epoch'], results['recall@k'], label='recall@k')
plt.plot(results['epoch'], results['map@k'], label='map@k')
plt.plot(results['epoch'], results['mrr@k'], label='mrr@k')
plt.plot(results['epoch'], results['ndcg@k'], label='ndcg@k')
plt.xlabel('Epoch')
plt.ylabel('Performance')
plt.title('Evaluation metrics')
plt.legend()
plt.show()